# 基本概念

## 统计语言模型

$p(s) = p(w_1) * p(w_2|w_1) * p(w_3|w_0w_1) * …… * p(w_n|w_1w_2……w_{n-1})$

说明:
+ wi 可以是字、词、短语或词类等等，称为统计基元。通常以“词”代之。
+ wi 的概率由 w1, ..., wi-1 决定，由特定的一组w1, ..., wi-1 构成的一个序列，称为 wi 的历史。

历史的运用：模型中有 $L^{i-1}$ 个参数 $p(wi|w_1...w_{i-1})$

其中 $L$ 表示共有 $L$ 个不同的基元（如有 $L$个不同的词）

指数型增长 $\rightarrow$ 爆炸

解决方案：只考虑固定长度的滑动窗口，即在长度为 $n$ 内若 $(w_{i-n+1},……,w_{i-1}) = (v_{k-n+1},……,v_{k-1})$ ，则认为，若 $w_{i} = v_{k}$，则 $p(w_i|S(w_{i-n+1},……,w_{i-1})) = p(v_k|S(v_{k-n+1},……,v_{k-1}))$

# n 元文法模型（n-gram）

**给定句子**：John read a book

**增加标记**：\<BOS\> John read a book\<EOS\>

**Unigram(1-gram)**: \<BOS\>, John, read, a, book, \<EOS\>

**Bigram(2-gram)**:(\<BOS\>John), (John read), (read a),(a book), (book \<EOS\>)

**Trigram(3-gram)**:(\<BOS\>John read), (John read a),(read a book), (a book \<EOS\>)

**定义**：$p(s) = \prod_i^{m+1} p(w_i|w_{i-n+1},……,w_{i-1})$

$w_0$ 为 \<BOS\>，$w_{m+1}$ 为 \<EOS\>。

**实现代码：**

In [2]:
from sympy import Segment


def create_ngram_model(text,n):
    words = text.split()
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = words[i: i + n]
        ngrams.append(ngram)
    return ngrams

# main:
n = 3
text = "<BOS> John read a book <EOS>"
print(create_ngram_model(text,n))

[['<BOS>', 'John', 'read'], ['John', 'read', 'a'], ['read', 'a', 'book'], ['a', 'book', '<EOS>']]


## 应用示例

### 1.音字转换问题

输入拼音串，输出可能的句子

$$
\begin{aligned}
\hat{CString} &= \arg\max\limits_{CString} p(CString \mid Pinyin) \\
&= \arg\max\limits_{CString} \frac{p(Pinyin \mid CString) \times p(CString)}{p(Pingyin)} \\
&= \arg\max\limits_{CString} p(Pinyin \mid CString) \times p(CString) \\
&= \arg\max\limits_{CString} p(CString)
\end{aligned}
$$

**解释：** 前面是条件概率基本应用，化简时 Pingyin 的消除其实是因为：**只要找最合适的，不需要真的算出概率**

(1)当前给的 Pingyin 已经固定，我们只关注谁的概率更高，因此相当于比较分子大小

(2)$p(Pingyin|Cstring) = 1$ ，这是显然的，Cstring 固定了就可以唯一确定 Pingyin，而计算时我们只找Pingyin符合的Cstring（实际上并非恒等变化，但还是同理，我们只想找出最合适的，那限定范围就在Pingyin符合的）

**结论：** 等于在语料库中出现的概率

### 2.汉语分词问题

给定汉字串：他是研究生物的，
可能的汉字串：
1）他|是|研究生|物|的
2）他|是|研究|生物|的

$$
\begin{align*}
\hat{Seg} &= \arg\max\limits_{Seg} p(Seg \mid Text) \\
&= \arg\max\limits_{Seg} \frac{p(Text \mid Seg) \times p(Seg)}{p(Text)} \\
&= \arg\max\limits_{Seg} p(Text \mid Seg) \times p(Seg) \\
&= \arg\max\limits_{Seg} p(Seg)
\end{align*}
$$

式中 $Text$ 是一个句子，$Seg$ 代表对 $Text$ 的划分

**实战：** 下面准备了相关的示例，先导入数据集，再运行计算代码

In [32]:
# 汉语分词问题

dataset_train_Chinese = [
    # === 场景一：地理与地标（用于强化“南京市”和“长江大桥”） ===
    "南京市 是 江苏省 的 省会",
    "宏伟 的 长江大桥 横跨 江面",
    "去 南京市 长江大桥 看 风景",
    "车辆 行驶 在 长江大桥 上",

    # === 场景二：职场与人物（用于制造“市长”和“江大桥”的歧义） ===
    # 设定：假设有一个叫“江大桥”的虚构人物，经常和“南京市长”一起出现
    "南京 市长 江大桥 发表 了 演讲",
    "江大桥 是 一位 勤奋 的 市长",
    "欢迎 南京 市长 来 参观",
]
dataset_test_divideChinese = [
    # 经典歧义：模型是切出 "长江大桥"(地标) 还是 "江大桥"(人名)？
    # 取决于你在训练集中喂了更多关于“桥梁”的数据，还是“市长”的数据
    "南京市长江大桥",

    # 边界测试
    "南京市长"
]

from collections import defaultdict

def create_ngram_model(text:str,n):
    words = text.split()
    words.insert(0,"<BOS>")
    words.append("<EOS>")
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = words[i: i + n]
        ngrams.append(ngram)
    return words,ngrams

def train_ngrams(n, dataset):
    # 若键值不存在，默认次数为0
    model = defaultdict(int)
    vocabulary = set()
    for text in dataset:
        words,ngrams = create_ngram_model(text,n)
        for word in words:
            vocabulary.add(word)
        for ngram in ngrams:
            # 字典的键值必须不可变，这里转化成元组
            # 1. ngram出现的次数
            ngram_tuple = tuple(ngram)
            model[ngram_tuple] += 1

            # 2. 前置n-1出现的次数
            context_tuple = ngram_tuple[:-1]
            model[context_tuple] += 1
    return vocabulary,model

def getSegments(maxlen,text:str,segment:list[str]):
    global segmentSet
    if len(text) == 0:
        segment.insert(0,"<BOS>")
        segment.append("<EOS>")
        segmentSet.append(segment)
        return
    for i in range(min(len(text),maxlen)):
        getSegments(maxlen,text[(i+1):],segment + [text[:(i+1)]])

if __name__ == '__main__':
    n = 2   # n元文法模型：n-gram
    m = 4   # 单词长度切分

    vocabulary,model = train_ngrams(n,dataset_train_Chinese)

    print("solution begin!")

    for text in dataset_test_divideChinese:
        # 初始化
        p = 0.0
        ans = ["Error"]
        segmentSet = []

        # 获取所有可能切分
        getSegments(m,text,[])

        for segment in segmentSet:
            p_temp = 1.0
            for i in range(0,len(segment) - n + 1):
                t = tuple(segment[i:i + n])
                count_all = model[t[:-1]]
                count_now = model[t]
                if count_all != 0:
                    # 做加一平滑
                    p_temp *= (count_now + 1.0) / (count_all + len(vocabulary))
                    # 可以替换如果不做平滑处理的后果
                    # p_temp *= count_now / count_all
                else:
                    p_temp *= 0

            # 最优化记录
            if p_temp > p:
                p = p_temp
                ans = segment

        print(f"I think {text} is {ans[1:-1]} (P = {p})")

solution begin!
I think 南京市长江大桥 is ['南京市', '长江大桥'] (P = 0.00011200716845878135)
I think 南京市长 is ['南京', '市长'] (P = 0.00033602150537634406)


In [41]:
# 拼音转汉语问题
# 顺序的调换是想先做切分，再映射为汉语
# 学习的时候不仅生成n-gram，同时做拼音到汉语的对应dict
# 思路大体一致，vibe coding：

from collections import defaultdict

# === 1. 数据集 (汉字已分词, 拼音一一对应) ===
dataset_train = [
    # ==========================================
    # 场景一：“研究” (Verb) vs “烟酒” (Noun)
    # 目的：训练 P(研究|做/搞) 高， P(烟酒|喜欢/买) 高
    # ==========================================
    ("他 喜欢 烟酒", "ta xi huan yan jiu"),
    ("烟酒 不 分家", "yan jiu bu fen jia"),
    ("商店 卖 烟酒", "shang dian mai yan jiu"),
    ("远离 烟酒 危害", "yuan li yan jiu wei hai"),
    ("过度 沉迷 烟酒", "guo du chen mi yan jiu"),

    ("正在 做 研究", "zheng zai zuo yan jiu"),
    ("科学 研究 很 重要", "ke xue yan jiu hen zhong yao"),
    ("专心 搞 研究", "zhuan xin gao yan jiu"),
    ("研究 成果 显著", "yan jiu cheng guo xian zhu"),
    ("他 在 研究所 工作", "ta zai yan jiu suo gong zuo"), # 引入"研究所"词汇

    # ==========================================
    # 场景二：“研究生” (Noun) vs “研究 生物” (Verb + Noun)
    # 目的：这是最难的歧义。
    # 如果后面跟的是“毕业/考试”，应该切成“研究生”。
    # 如果后面跟的是“的/特性”，且前面是动词语境，可能切成“研究 生物”。
    # ==========================================
    # 强化 "研究生" (作为一个整体词)
    ("他 是 研究生", "ta shi yan jiu sheng"),
    ("研究生 毕业 了", "yan jiu sheng bi ye le"),
    ("报考 研究生", "bao kao yan jiu sheng"),
    ("学历 是 研究生", "xue li shi yan jiu sheng"),
    ("读 研究生 很 累", "du yan jiu sheng hen lei"),

    # 强化 "研究" + "生物" (Bigram)
    ("研究 生物 的 构造", "yan jiu sheng wu de gou zao"),
    ("他 想 研究 生物", "ta xiang yan jiu sheng wu"),
    ("我们 研究 生物 钟", "wo men yan jiu sheng wu zhong"),
    ("生物 是 一门 学科", "sheng wu shi yi men xue ke"), # 强化“生物”作为独立词的频率
    ("海洋 生物 很多", "hai yang sheng wu hen duo"),

    # ==========================================
    # 场景三：“他是” (Subj+Verb) vs “踏实” (Adj)
    # 目的：区分 ta shi
    # ==========================================
    ("做事 要 踏实", "zuo shi yao ta shi"),
    ("踏实 肯干", "ta shi ken gan"),
    ("睡 得 很 踏实", "shui de hen ta shi"),
    ("做人 要 踏实", "zuo ren yao ta shi"),

    ("他 是 老师", "ta shi lao shi"),
    ("他 是 科学家", "ta shi ke xue jia"),
    ("她 是 医生", "ta shi yi sheng"), # 注意：拼音ta一样，汉字不同，增加候选词
    ("它 是 机器", "ta shi ji qi"),

    # ==========================================
    # 场景四：词汇覆盖 (防止出现 Unknown 字导致概率断裂)
    # ==========================================
    ("无 忧 无 虑", "wu you wu lv"), # 覆盖 "wu" -> 无
    ("前面 有 大雾", "qian mian you da wu"), # 覆盖 "wu" -> 雾
    ("不仅 如此", "bu jin ru ci"),
    ("的", "de"), # 这是一个Trick，保证常用虚词在词典里
    ("了", "le")
]

# === 2. 自动训练：同时学习“词典”和“概率” ===
def train_model_from_corpus(dataset):
    # 词典：Key=拼音串, Value=可能的汉字词集合
    # e.g. "yan jiu" -> {"研究", "烟酒"}
    pinyin_to_word = defaultdict(set)

    # 概率统计
    word_bigram_counts = defaultdict(int)
    word_unigram_counts = defaultdict(int)
    vocab_size = 0

    for hanzi_str, pinyin_str in dataset:
        h_words = hanzi_str.split()  # ["他", "是", "研究生"]
        p_tokens = pinyin_str.split() # ["ta", "shi", "yan", "jiu", "sheng"]

        # --- 核心逻辑：自动对齐提取词典 ---
        p_cursor = 0 # 拼音游标

        # 加上 BOS/EOS 用于语言模型训练
        training_words = ["<BOS>"] + h_words + ["<EOS>"]

        # 1. 遍历汉字词，切取对应的拼音
        for word in h_words:
            # 获取当前词的字数，比如 "研究生" 长度为 3
            word_len = len(word)

            # 从拼音列表中切出对应长度的片段
            # 比如 cursor=2, len=3, 切出 p_tokens[2:5] -> ["yan", "jiu", "sheng"]
            current_pinyin_segment = p_tokens[p_cursor : p_cursor + word_len]

            # 拼合成字符串作为 Key
            pinyin_key = " ".join(current_pinyin_segment)

            # 存入词典：学会了这个拼音串对应这个词
            pinyin_to_word[pinyin_key].add(word)

            # 移动游标
            p_cursor += word_len

        # 2. 统计 N-gram 概率 (和之前一样)
        for i in range(len(training_words)-1):
            w_curr = training_words[i]
            w_next = training_words[i+1]
            word_unigram_counts[w_curr] += 1
            word_bigram_counts[(w_curr, w_next)] += 1

        vocab_size += len(training_words)

    return pinyin_to_word, word_bigram_counts, word_unigram_counts, vocab_size

# === 3. 拼音切分与解码 (Lattice 生成) ===
# 这里逻辑不变：尝试切分输入拼音，看词典里有没有
def solve_pinyin_lattice(pinyin_list, current_words, results, dictionary):
    if not pinyin_list:
        results.append(current_words)
        return

    max_search_len = 4 # 假设最长词只有4个字，防死循环

    # 尝试切出前 k 个拼音
    for k in range(1, min(len(pinyin_list) + 1, max_search_len + 1)):
        segment = pinyin_list[:k]
        segment_key = " ".join(segment)

        # 如果这个拼音串在刚才自动学习的词典里存在
        if segment_key in dictionary:
            possible_words = dictionary[segment_key]
            for word in possible_words:
                # 递归继续切分剩余部分
                solve_pinyin_lattice(pinyin_list[k:], current_words + [word], results, dictionary)

# === 4. 计算句子概率 ===
def get_score(sentence, bi_counts, uni_counts, v_size):
    prob = 1.0
    full = ["<BOS>"] + sentence + ["<EOS>"]
    for i in range(len(full)-1):
        pre = full[i]
        cur = full[i+1]
        # 加一平滑
        prob *= (bi_counts[(pre, cur)] + 1) / (uni_counts[pre] + v_size)
    return prob

# === 主程序 ===
if __name__ == '__main__':
    # 1. 自动从数据中学习
    print("正在从语料库提取词典并训练模型...")
    p2w_dict, bi_counts, uni_counts, v_size = train_model_from_corpus(dataset_train)

    # 打印一下自动学到的词典看看
    print("--- 自动提取的词典 (部分) ---")
    for py, words in list(p2w_dict.items())[:5]:
        print(f"[{py}] -> {words}")

    # 2. 测试输入
    # 这是一个很有趣的测试：它既可以是 "研究 生物", 也可以是 "研究生 (wu?)"
    # 但因为我们只学过 "wu"->"无"，也学过 "sheng wu"->"生物"
    pinyinset = ["yan jiu sheng wu","ta xi huan yan jiu","ta zai yan jiu sheng wu" ,"yan jiu sheng bi ye le"]
    for target_pinyin in pinyinset:
        print(f"--- 开始解码: {target_pinyin} ---")
        candidates = []
        solve_pinyin_lattice(target_pinyin.split(), [], candidates, p2w_dict)

        # 3. 寻找最优解
        best_sent = []
        best_score = -1

        for sent in candidates:
            score = get_score(sent, bi_counts, uni_counts, v_size)
            sent_str = " ".join(sent)
            print(f"候选: {sent_str} \t(P={score:.6e})")
            if score > best_score:
                best_score = score
                best_sent = sent_str

        print(f"✅ 最终识别结果:【{best_sent}】")

正在从语料库提取词典并训练模型...
--- 自动提取的词典 (部分) ---
[ta] -> {'她', '它', '他'}
[xi huan] -> {'喜欢'}
[yan jiu] -> {'烟酒', '研究'}
[bu] -> {'不'}
[fen jia] -> {'分家'}
--- 开始解码: yan jiu sheng wu ---
候选: 烟酒 生物 	(P=6.760411e-07)
候选: 研究 生物 	(P=4.009623e-06)
候选: 研究生 无 	(P=3.440209e-07)
✅ 最终识别结果:【研究 生物】
--- 开始解码: ta xi huan yan jiu ---
候选: 她 喜欢 烟酒 	(P=1.647946e-08)
候选: 她 喜欢 研究 	(P=6.108765e-09)
候选: 它 喜欢 烟酒 	(P=1.647946e-08)
候选: 它 喜欢 研究 	(P=6.108765e-09)
候选: 他 喜欢 烟酒 	(P=1.120222e-07)
候选: 他 喜欢 研究 	(P=4.152548e-08)
✅ 最终识别结果:【他 喜欢 烟酒】
--- 开始解码: ta zai yan jiu sheng wu ---
候选: 她 在 烟酒 生物 	(P=2.395270e-11)
候选: 她 在 研究 生物 	(P=9.470954e-11)
候选: 她 在 研究生 无 	(P=1.218895e-11)
候选: 它 在 烟酒 生物 	(P=2.395270e-11)
候选: 它 在 研究 生物 	(P=9.470954e-11)
候选: 它 在 研究生 无 	(P=1.218895e-11)
候选: 他 在 烟酒 生物 	(P=1.628230e-10)
候选: 他 在 研究 生物 	(P=6.438059e-10)
候选: 他 在 研究生 无 	(P=8.285668e-11)
✅ 最终识别结果:【他 在 研究 生物】
--- 开始解码: yan jiu sheng bi ye le ---
候选: 研究生 毕业 了 	(P=2.457292e-08)
✅ 最终识别结果:【研究生 毕业 了】


## n 元语法模型的获取

**训练语料：** 用于建立模型，确定模型参数的己知语料

**最大似然估计：** 可以得出，用相对频率计算概率的方法，概率最大

$$p(w_i \mid w_{i-n+1}^{i-1}) = f(w_i \mid w_{i-n+1}^{i-1}) = \frac{c(w_{i-n+1}^i)}{\sum_{w_i} c(w_{i-n+1}^i)}$$

其中 $c(w^j_i)$ 表示语料库中 $k$ 从 $i$ 到 $j$ 分别为 $w_k$ 的情况的计数，$\sum_{w_i} c(w_{i-n+1}^i)$ 实际上是前 $n-1$ 符合的情况下，最后一个可能的所有情况计数

## 数据平滑

**基本思想：** 劫富济贫，0 和 1 差太多，减少正确和错误之前的差异

**基本约束：**

$$
\sum_{w_i} p(w_i \mid w_1, w_2, \cdots, w_{i-1}) = 1
$$

**方法：**

加 1 法： 对所有情况出现次数 + 1

\begin{align*}
p(w_i \mid w_{i-n+1}…… w_{i-1}) &= \frac{1 + c(w_{i-n+1}……w_{i-1}w_i)}{\sum_{w_i} \left[1 + c(w_{i-n+1}……w_{i-1}w_i)\right]} \\
                    &= \frac{1 + c(w_{i-n+1}……w_{i-1}w_i)}{|V| + \sum_{w_i} c(w_{i-n+1}……w_{i-1}w_i)}
\end{align*}

其中 $|V|$ 是所有可能的的词汇量（其实此处并非严格等于，但是实际应用中差别不大）

<mark>\<BOS\> \<EOS\> 不需要计入 $|V|$ </mark>，但这到底是为什么呢？

# n元模型应用

## 采用基于语言模型的分词方法

这一部分笔者觉得相当抽象，个人（~~AI~~）理解要解决的问题是：
很多词（如人名、地名、具体的时间“3月14日”）在训练语料里根本没出现过，概率为0。如果把所有可能的数字、人名都存进词表，词表会无限大。

因此用一些标签来代替，定义了四类：
+ **分词词典中规定的词**;
+ **由词法规则派生出来的词或短语**，如:干干净净、非党员、全面性、检查员、看不出、克服了、走出来...
+ **与数字相关的实体**，如:日期、时间、货币、百分数、温度、长度、面积、重量、电话号码、邮件地址等;
+ **专有名词**，如:人名（PN）、地名（LP）、机构名（ON）。

**规定：** PN，LP，ON 分别单独为一类，实体FT（包括dat、tim、per、mon等等）作为一个类

$$
\begin{align*}
\hat{C} &= \arg\max\limits_{C} p(C \mid S) \\
&= \arg\max\limits_{C} \frac{p(C) \times p(S \mid C)}{p(S)} \\
&\stackrel{归一化}= \arg\max\limits_{C} p(C) \times p(S \mid C)
\end{align*}
$$

**语言模型** $p(C)$ 可采用三元语法，其实就是把之前的 $Seg、Word$ 换成 $Class$

**生成模型** $p(S\mid C) \approx \prod_{i=1}^N p(s_i|c_i)$

**含义：** 已知是哪一类，生成对应汉字串 s 的概率

计算如下：

1. 若 C 为 FT、LW（Lexicon Word词表词）、MW（Morphological Word词法派生词）查对应的表，有则 $P(s|c) = 1$，否则为 $0$
2. PN、LN 均为基于字的 2 元模型
3. ON 为基于词的 2 元模型

所谓基于字的 2 元模型是一个子模型。

比如计算“比尔盖茨”是人名的概率，要看“比-尔”、“尔-盖”、“盖-茨”这些字在人名库中相邻出现的概率。

**训练算法**

这是一个“鸡生蛋，蛋生鸡”的问题：只有分好词才能训练模型，但只有有了模型才能分好词。

+ 初始化：先用基础分词器切一版粗糙的语料。
+ M步 (Maximization)：用这一版语料统计参数，算出公式(3)中的概率。
+ E步 (Expectation)：用算出的新模型，重新对语料进行更准确的切分。
+ 循环：重复2-3步，直到结果稳定。

**分词和词性判断一体化：**

核心思想：二者信息可以相互辅助

**子模型 1：基于词性的生成模型**
$$ P(W, T) \approx \prod P(w_i \mid t_i) \times P(t_i \mid t_{i-1}, t_{i-2}) $$
*   **物理意义**：这就是标准的HMM（隐马尔可夫模型）。
    *   $P(t_i \mid t_{i-1}, t_{i-2})$：词性转移概率（如：名词后面接动词的概率）。
    *   $P(w_i \mid t_i)$：发射概率（如：已知是动词，这个词是“写”的概率）。

**子模型 2：基于单词的统计模型**
$$ P(W, T) \approx \prod P(t_i \mid w_i) \times P(w_i \mid w_{i-1}, w_{i-2}) $$
*   **物理意义**：
    *   $P(w_i \mid w_{i-1}, w_{i-2})$：标准的词汇3-gram（如：“文章”后面接“写”的概率）。
    *   $P(t_i \mid w_i)$：这个词本身具有某个词性的概率（如：“写”这个字由90%概率是动词，10%是名词）。

**综合公式**：
$$ P^*(W, T) = \alpha \times [\text{子模型1}] + \beta \times [\text{子模型2}] $$
*   通过参数 $\alpha$ 和 $\beta$ 来调节权重。
    *   如果数据稀疏（生僻词多），子模型1（基于词性）更可靠，因为词性转移规律很稳定。
    *   如果常见词多，子模型2（基于词汇）更准确，因为它捕捉了具体的习惯用语。

In [ ]:
# 待实现
# 困难的
# 有空再说

# n元模型的问题

该 课程 很 <span style="color:purple">枯燥</span>，大家 觉得 很 <span style="color:purple">无聊</span>。


$$
P(\text{无聊}|\text{很}) = \frac{\text{count(很 无聊)}}{\text{count(很)}}
$$


## 问题①：数据稀疏
N-元组“很 无聊”未出现过，则回退


## 问题②：忽略语义相似性
“无聊”与“枯燥”虽语义相似，但无法共享信息

## 解决方案：基于连续语义空间的词语表示

简单来说就是，原来是采用独热编码，即在模型眼里：

$$向量(枯燥)×向量(无聊)=0$$

但这俩是近义词，因此需要用稠密的实数向量代替

通过模型学习，可以把语义相近的放在相近的空间，使用时会表现出和近义词相似的规律

# 语言模型的发展

+ 阶段一（约1970之前），基于规则的语言模型：主要基于手写规则。
+ 阶段二（约1970-约2000），基于统计的语言模型：从数学统计的角度预测下个词的出现概率，如N-Gram等，推理过程非常直观，但是推理结果非常受数据集的影响，容易出现数据稀疏（即空值）等问题。
+ 阶段三（约2000-约2017），基于神经网络的语言模型：比如NNLM、RNN、LSTM等。
+ 阶段四（约2018-现在），基于Transformer的语言模型：比如BERT、GPT系列等大模型。

宗成庆:《自然语言处理》

谢谢！